In [1]:
# S&P 500 Futures Price Calculation

# Given information
spot_price = 5700  # Current S&P 500 index value
fed_funds_rate = 0.05  # 500 bps = 5%
dividend_yield = 0.0122  # SPY yield of 1.22%
days_to_expiration = 75  # Approximate days from now to mid-December

# Calculate the futures price
risk_free_rate = fed_funds_rate  # Assuming fed funds rate as risk-free rate
time_fraction = days_to_expiration / 365  # Fraction of a year

futures_price = spot_price * (1 + (risk_free_rate - dividend_yield) * time_fraction)

print(f"Calculated futures price: {futures_price:.2f}")
print(f"Actual December contract price: 5749.00")
print(f"Difference: {5749 - futures_price:.2f}")

# Additional factors to consider
print("\nFactors that might explain the difference:")
print("1. Market expectations for index growth")
print("2. Supply and demand dynamics in the futures market")
print("3. Differences in borrowing costs among market participants")
print("4. Variations in dividend forecasts")
print("5. Risk premium for holding the futures contract")

Calculated futures price: 5744.27
Actual December contract price: 5749.00
Difference: 4.73

Factors that might explain the difference:
1. Market expectations for index growth
2. Supply and demand dynamics in the futures market
3. Differences in borrowing costs among market participants
4. Variations in dividend forecasts
5. Risk premium for holding the futures contract
